In [2]:
!pip install ultralytics

In [3]:
import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.0.131 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/166.8 GB disk)


# YOLOv8 custom dataset 학습시키기

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import zipfile
import shutil
import os
import glob
import random

from datetime import datetime

# 1.데이터셋 준비

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
data_path = "/content/gdrive/MyDrive/"

ROOT_DIR = "/content"

DATA_ROOT_DIR = os.path.join(ROOT_DIR, "damage_detection")

TRAIN_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "train")
VALID_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "valid")

In [7]:
shutil.copy(os.path.join(data_path, "damage_train_images_35000_v2.zip"), ROOT_DIR)

'/content/damage_train_images_35000_v2.zip'

In [8]:
shutil.copy(os.path.join(data_path, "damage_train_labels_35000_v2.zip"), ROOT_DIR)

'/content/damage_train_labels_35000_v2.zip'

In [9]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, "damage_train_images_35000_v2.zip")) as target_file:
  target_file.extractall(TRAIN_DATA_ROOT_DIR)
  print("success!")

success!


In [10]:
old_folder_name = os.path.join(TRAIN_DATA_ROOT_DIR, "damage")

# 새로운 폴더 이름
new_folder_name = os.path.join(TRAIN_DATA_ROOT_DIR, "images")

# 폴더 이름 재설정
os.rename(old_folder_name, new_folder_name)


In [11]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, "damage_train_labels_35000_v2.zip")) as target_file:
  target_file.extractall(TRAIN_DATA_ROOT_DIR)
  print("success!")


success!


In [12]:
TRAIN_DATA_IMAGE_ROOT_DIR = os.path.join(TRAIN_DATA_ROOT_DIR, "images")
TRAIN_DATA_LABEL_ROOT_DIR = os.path.join(TRAIN_DATA_ROOT_DIR, "labels")

VALID_DATA_IMAGE_ROOT_DIR = os.path.join(VALID_DATA_ROOT_DIR, "images")
VALID_DATA_LABEL_ROOT_DIR = os.path.join(VALID_DATA_ROOT_DIR, "labels")

## 1-1. train용 img/txt파일의 갯수가 일치하는지 확인

In [13]:
print("train/images 폴더의 파일 갯수: ", len(os.listdir(TRAIN_DATA_IMAGE_ROOT_DIR)))   # (train/images) 파일의 갯수: 508964
print("train/labels 폴더의 파일 갯수: ", len(os.listdir(TRAIN_DATA_LABEL_ROOT_DIR)))   # (valid/labels)  파일의 갯수: 508964

image_file_set = ()        # 집합현산을 위해 빈 set() 만들어주기
label_file_set = ()

image_file_name_list = []
label_file_name_list = []

for file in os.listdir(TRAIN_DATA_IMAGE_ROOT_DIR):
  image_file_name_list.append(os.path.splitext(file)[0])   # [0426795_as-0099618, img] 에서 [0]이면 이름 부분

for file in os.listdir(TRAIN_DATA_LABEL_ROOT_DIR):
  label_file_name_list.append(os.path.splitext(file)[0])   # [0426795_as-0099618, txt] 에서 [0]이면 이름 부분

image_file_set = set(image_file_name_list)
label_file_set = set(label_file_name_list)
print("동일한 이름을 갖는 img파일과 txt파일의 갯수: ", len(image_file_set & image_file_set))

train/images 폴더의 파일 갯수:  33713
train/labels 폴더의 파일 갯수:  33713
동일한 이름을 갖는 img파일과 txt파일의 갯수:  33713


In [14]:
shutil.copy(os.path.join(data_path, "damage_valid_images_35000_v2.zip"), ROOT_DIR)

'/content/damage_valid_images_35000_v2.zip'

In [15]:
shutil.copy(os.path.join(data_path, "damage_valid_labels_35000_v2.zip"), ROOT_DIR)

'/content/damage_valid_labels_35000_v2.zip'

In [16]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, "damage_valid_images_35000_v2.zip")) as target_file:
  target_file.extractall(VALID_DATA_ROOT_DIR)
  print("success!")

success!


In [17]:
with zipfile.ZipFile(os.path.join(ROOT_DIR, "damage_valid_labels_35000_v2.zip")) as target_file:
  target_file.extractall(VALID_DATA_ROOT_DIR)
  print("success!")

success!


## 1-2. valid용 img/txt 파일의 갯수가 일치하는지 확인

In [18]:
print("valid/images 폴더의 파일 갯수: ", len(os.listdir(VALID_DATA_IMAGE_ROOT_DIR)))   # (valid/images) 파일의 갯수: 101792
print("valid/labels 폴더의 파일 갯수: ", len(os.listdir(VALID_DATA_LABEL_ROOT_DIR)))   # (valid/labels)  파일의 갯수: 101792

image_file_set = ()        # 집합연산을 위해 빈 set() 만들어주기
label_file_set = ()

image_file_name_list = []
label_file_name_list = []

for file in os.listdir(VALID_DATA_IMAGE_ROOT_DIR):
  image_file_name_list.append(os.path.splitext(file)[0])   # [0426795_as-0099618, img] 에서 [0]이면 이름 부분

for file in os.listdir(VALID_DATA_LABEL_ROOT_DIR):
  label_file_name_list.append(os.path.splitext(file)[0])   # [0426795_as-0099618, txt] 에서 [0]이면 이름 부분

image_file_set = set(image_file_name_list)
label_file_set = set(label_file_name_list)
print("동일한 이름을 갖는 img파일과 txt파일의 갯수: ", len(image_file_set & image_file_set))

valid/images 폴더의 파일 갯수:  5828
valid/labels 폴더의 파일 갯수:  5828
동일한 이름을 갖는 img파일과 txt파일의 갯수:  5828


# 2.YOLOv8에 필요한 모듈 설치

##

In [19]:
!pip install PyYAML

## 2-2. yaml 파일 준비

In [20]:
import yaml

data = {
    'train': os.path.join(DATA_ROOT_DIR, "train/images"),
    'val': os.path.join(DATA_ROOT_DIR, "valid/images"),
    'nc': 4,
    'names': ['Crushed', 'Scratched', 'Breakage','Separated']
}

# data.yaml 파일로 저장
with open('/content/damage_detection/data.yaml', 'w') as file:
    yaml.dump(data, file)

## 2-3. 가중치 파일 준비

In [21]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:00<00:00, 64.8MB/s]


# 3.학습

In [ ]:
model.train(data = "/content/damage_detection/data.yaml", epochs=32, patience=30, batch=32, imgsz=512)

Ultralytics YOLOv8.0.131 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/damage_detection/data.yaml, epochs=32, patience=30, batch=32, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fals

# 4.예측

In [ ]:
if not os.path.exists("/content/test/images"):
  os.mkdir("/content/test/images")

## 4-1.폴더 만들고 test에 쓸 사진 넣어주기

In [ ]:
result = model.predict(source = "/content/test/images/", save=True)